In [1]:
import bs4
import pandas as pd
from urllib import request
import re
import matplotlib.pyplot as plt
import numpy as np
import ast
import sklearn

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics

In [23]:
df_series=pd.read_csv('C:/Users/Jean-Baptiste/OneDrive/ENSAE/projet_python_data_reg.csv')


In [4]:
#on élimine la colonne Unnamed et on remet Genre sous forme de liste
df_series=df_series.drop(['Unnamed: 0'],axis=1)
df_series['Genre']=df_series['Genre'].apply(ast.literal_eval)

In [5]:
1 in df_series['series_anthologique']

True

In [6]:
def row_value(df, column, value):
    L=[]
    for k in range(len(df)):
        if df.loc[k,column]==value:
            L.append(k)
    return(L)


On aura besoin de la durée et de la saison. Cependant en raison du nombre important de pays qui n'ont qu'une série dans ce classement, et du fait que cette variable ne se base que sur l'origine de la plateforme de diffusion, ce qui la rend assez douteuse, on ne l'incluera pas dans la régression.

In [7]:
df_series_reg=df_series.copy()
df_series_reg=df_series_reg.drop(labels=row_value(df_series, 'Durée', 'Inconnue'))
df_series_reg['Durée']=df_series_reg['Durée'].astype('float')

In [8]:
#pour faire une régression sur une variable non numérique, on crée des variable binaires associées aux valeurs de la variables d'origine
for saison in ['printemps','été','automne','hiver']:
    df_series_reg[saison+'_bin']=df_series_reg['Saison'].apply(lambda x: float(x==saison))

In [9]:
df_series_reg.columns

Index(['Nom', 'Note', 'Auteur', 'Genre', 'Nombre de saisons', 'Plateforme',
       'Durée', 'Date de sortie', 'Mois', 'Année', 'Saison', 'Pays',
       'series_policier', 'series_drame', 'series_thriller',
       'series_épouvante-horreur', 'series_mini-série', 'series_guerre',
       'series_action', 'series_historique', 'series_aventure',
       'series_fantasy', 'series_comédie', 'series_animation',
       'series_jeunesse', 'series_documentaire', 'series_dramatique',
       'series_romance', 'series_shōnen', 'series_seinen',
       'series_fantastique', 'series_science-fiction', 'series_sport',
       'series_biopic', 'series_musique', 'series_anthologique',
       'series_judiciaire', 'series_western', 'series_médical',
       'series_télé-réalité', 'printemps_bin', 'été_bin', 'automne_bin',
       'hiver_bin'],
      dtype='object')

On doit diviser le df en deux sous df, le 1er servira à l'entrainement du modèle, le second à la prédiction. Les deux sont formés aléatoirement. 

In [10]:
df_series_reg1=df_series_reg[['Note','Durée','series_policier', 'series_drame', 'series_thriller',
       'series_épouvante-horreur', 'series_mini-série', 'series_guerre',
       'series_action', 'series_historique', 'series_aventure',
       'series_fantasy', 'series_comédie', 'series_animation',
       'series_jeunesse', 'series_documentaire', 'series_dramatique',
       'series_romance', 'series_shōnen', 'series_seinen',
       'series_fantastique', 'series_science-fiction', 'series_sport',
       'series_biopic', 'series_musique', 'series_anthologique',
       'series_judiciaire', 'series_western', 'series_médical',
       'series_télé-réalité', 'printemps_bin', 'été_bin', 'automne_bin',
       'hiver_bin']]

In [11]:
# #on divise l'échantillon entre la partie qui va entrainer le modèle et celle qui devra le valider
X_train, X_test, y_train, y_test = train_test_split(
    df_series_reg1.drop(['Note'], axis = 1),
    df_series_reg1[['Note']].values.ravel(), test_size=0.2, random_state=0)

#on entraine notre modèle
ols = LinearRegression().fit(X_train, y_train)
y_pred= ols.predict(X_test)

In [12]:
print(ols.intercept_, ols.coef_)

6.6429531183454715 [ 0.00436189 -0.09770576  0.04409422  0.14177654 -0.32584413  0.27756635
  0.39075759 -0.19727762  0.23218759 -0.00224524 -0.23999878 -0.00511659
  0.40452802  0.18665319  0.92331016 -0.31324095 -0.32116194  0.60532482
  0.49422076  0.01127261 -0.30672901  0.56298259 -0.52461788 -0.02254473
  0.94912489  0.46230816  0.76901253  0.31355242 -0.50265709  0.14652745
 -0.11442757 -0.12753519  0.09543532]


In [13]:
var=['Durée','series_policier', 'series_drame', 'series_thriller',
       'series_épouvante-horreur', 'series_mini-série', 'series_guerre',
       'series_action', 'series_historique', 'series_aventure',
       'series_fantasy', 'series_comédie', 'series_animation',
       'series_jeunesse', 'series_documentaire', 'series_dramatique',
       'series_romance', 'series_shōnen', 'series_seinen',
       'series_fantastique', 'series_science-fiction', 'series_sport',
       'series_biopic', 'series_musique', 'series_anthologique',
       'series_judiciaire', 'series_western', 'series_médical',
       'series_télé-réalité', 'printemps_bin', 'été_bin', 'automne_bin',
       'hiver_bin']

In [14]:
for k in range(len(var)):
    print('le coefficient de la régression associée à '+var[k]+' est %.2f'
          % ols.coef_[k])

le coefficient de la régression associée à Durée est 0.00
le coefficient de la régression associée à series_policier est -0.10
le coefficient de la régression associée à series_drame est 0.04
le coefficient de la régression associée à series_thriller est 0.14
le coefficient de la régression associée à series_épouvante-horreur est -0.33
le coefficient de la régression associée à series_mini-série est 0.28
le coefficient de la régression associée à series_guerre est 0.39
le coefficient de la régression associée à series_action est -0.20
le coefficient de la régression associée à series_historique est 0.23
le coefficient de la régression associée à series_aventure est -0.00
le coefficient de la régression associée à series_fantasy est -0.24
le coefficient de la régression associée à series_comédie est -0.01
le coefficient de la régression associée à series_animation est 0.40
le coefficient de la régression associée à series_jeunesse est 0.19
le coefficient de la régression associée à seri

On observe que:

La note varie autour de 6.64 (terme constant), ce qui reflète les notes assez élevées des séries du df

La durée a une influence presque négligeable sur la note (inférieur à 0.01).

Le genre a des effets différents: les genres épouvante-horreur, télé-réalité et biopic semblent être assez pénalisant pour la note tandis qu'être un western, un shounen ou une anthologie bonnifient grandement la note. Cependant on remarque que les genres ayant le plus d'impact au niveau de la régression sont aussi ceux apparaissant le moins. Il faut donc être prudent quant aux conclusions que cela peut induire.

Au contraire, les genres les plus répandus (comédie, drame) ont une influence absolue négligeable quant à la note.

Entre ces deux états, on observe des genres représentés mais non surreprésentés, dont le coefficient dans la régression est non négligeable comme l'animation, l'action et le thriller, sans être aussi grand (en va) que ceux des critères 'rares'.

La saison de diffusion a une influence plus faible que le(s) genre(s) d'une série mais non négligeable. Il apparait dans ce sens que les séries sortie en hiver et printemps sont mieux appéciées que celles qui sortent en été/automne 

In [15]:
rmse = sklearn.metrics.mean_squared_error(y_test, y_pred)
rsq = sklearn.metrics.r2_score(y_test, y_pred) 

print('Mean squared error: %.2f'
      % rmse)
print('Coefficient of determination: %.2f'
      % rsq)
      

Mean squared error: 0.58
Coefficient of determination: 0.21


Cependant cela fait beucoup de variable pour une régression, on peut retirer la date (qui n'a presque aucune influence comme on vient de le voir). On peut essayer de retirer également les variables de genres qui n'apparaissent pas beaucoup dans l'échantillon de séries ce qui peut biaiser leur importance dans la note. 

Que se passe-t-il si on retire les genres les plus rares ? (ce qui peut être fait puisque toutes les séries auront au moins un autre genre pour déterminer la note)

In [16]:
#liste des genres rares
genres_r=['télé-réalité','médical','western', 'judiciaire', 'biopic', 'sport', 'musique','anthologique']
labels=['series_'+k for k in genres_r]

In [17]:
df_series_reg3=df_series_reg1.copy()
df_series_reg3=df_series_reg3.drop(labels, axis=1)
df_series_reg3=df_series_reg3.drop('Durée', axis=1)


In [18]:
var2=df_series_reg3.columns[1:]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df_series_reg3.drop(['Note'], axis = 1),
    df_series_reg3[['Note']].values.ravel(), test_size=0.2, random_state=0)

ols = LinearRegression().fit(X_train, y_train)
y_pred = ols.predict(X_test)

In [20]:
print(ols.intercept_, ols.coef_)

6.859964006897895 [-0.0867424   0.05182416  0.22996752 -0.28572463  0.25917686  0.41953594
 -0.20179283  0.17113166 -0.020461   -0.25731633 -0.0763904   0.33417027
  0.1729281   0.84039822 -0.33357361 -0.34273199  0.5596293   0.44972813
 -0.00325573 -0.28638032  0.12247697 -0.1270712  -0.12419993  0.12879415]


In [21]:
for k in range(len(var2)):
    print('le coefficient de la régression associée à '+var2[k]+' est %.2f'
          % ols.coef_[k])

le coefficient de la régression associée à series_policier est -0.09
le coefficient de la régression associée à series_drame est 0.05
le coefficient de la régression associée à series_thriller est 0.23
le coefficient de la régression associée à series_épouvante-horreur est -0.29
le coefficient de la régression associée à series_mini-série est 0.26
le coefficient de la régression associée à series_guerre est 0.42
le coefficient de la régression associée à series_action est -0.20
le coefficient de la régression associée à series_historique est 0.17
le coefficient de la régression associée à series_aventure est -0.02
le coefficient de la régression associée à series_fantasy est -0.26
le coefficient de la régression associée à series_comédie est -0.08
le coefficient de la régression associée à series_animation est 0.33
le coefficient de la régression associée à series_jeunesse est 0.17
le coefficient de la régression associée à series_documentaire est 0.84
le coefficient de la régression a

On remarque que la nouvelle régression a changé les coefficients de certains genres

Pour certains genres (policier, drame, guerre, science-fiction...), cette transformation a augmenté leur coefficient. Pour d'autres (documentaires, mini-série...), ça l'a abaissé.

Ainsi, bien que les variations ne dépassent pas souvent 0.1, on peut voir une influence combinée des genres pour déterminer la note, les genres que l'on a éliminé apparaissant toujours avec d'autres dans le descriptif d'une série.

In [22]:
rmse = sklearn.metrics.mean_squared_error(y_test, y_pred)
rsq = sklearn.metrics.r2_score(y_test, y_pred) 

print('Mean squared error: %.2f'
      % rmse)
print('Coefficient of determination: %.2f'
      % rsq)

Mean squared error: 0.61
Coefficient of determination: 0.17


On obtient une erreur quadratique légèrement plus importante, et un coefficient de détermination légèrement plus important, mais étant donné que les genres que l'on oublie ne comptent que pour un nombre faible de séries (pas plus de 10), le gain en temps de traitement est minime